In [18]:
# forest_or_age_race_fixed.py
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from pandas.api.types import CategoricalDtype

# ----------------------------
# Path resolution (root or notebooks/)
# ----------------------------
CWD = Path.cwd()
if (CWD / "data").exists():
    DATA_DIR = CWD / "data"
    FIGURE_DIR = CWD / "figures"
elif (CWD.name == "notebooks") and (CWD.parent / "data").exists():
    DATA_DIR = CWD.parent / "data"
    FIGURE_DIR = CWD.parent / "figures"
else:
    raise FileNotFoundError(
        "Could not find a 'data' folder in the current directory or its parent.\n"
        f"Checked: {CWD} and {CWD.parent}"
    )
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# Load data
# ----------------------------
csv_path = DATA_DIR / "processed_simulated_smoking_cessation_cohort.csv"
if not csv_path.exists():
    raise FileNotFoundError(f"Data file not found: {csv_path}")

df = pd.read_csv(csv_path)
print(f"✅ Loaded: {csv_path}")
print("📋 Columns:", df.columns.tolist())

# ----------------------------
# Filter to baseline smokers & normalize flags
# ----------------------------
if df["baseline_smoker"].dtype == "object":
    df["baseline_smoker"] = df["baseline_smoker"].map({"Yes": 1, "No": 0})

df_smokers = df[df["baseline_smoker"] == 1].copy()
if df_smokers.empty:
    raise ValueError("No baseline smokers found after filtering (baseline_smoker == 1).")

# ----------------------------
# intervention_group → force correct mapping
# (we know your labels are 'cessation program' vs 'usual care')
# ----------------------------
ig_raw = df_smokers["intervention_group"].astype("string").str.strip().str.lower()
mapping_forced = {
    "cessation program": 1,
    "usual care": 0
}
if set(ig_raw.dropna().unique()) <= set(mapping_forced.keys()):
    df_smokers["intervention_group"] = ig_raw.map(mapping_forced).astype("Int64")
    print("ℹ️ intervention_group mapping (forced):", mapping_forced)
else:
    # fallback: try a broad mapping, then auto-map any remaining 2 labels
    true_tokens = {
        "1","yes","y","true","t","intervention","treated","treatment",
        "program","enrolled","participant","participated","intervene","cessation program"
    }
    false_tokens = {
        "0","no","n","false","f","control","comparison","baseline","none",
        "not enrolled","did not participate","didn’t participate","didnt participate","usual care"
    }
    mapped = pd.Series(pd.NA, index=ig_raw.index, dtype="Int64")
    mapped = mapped.mask(ig_raw.isin(true_tokens), 1)
    mapped = mapped.mask(ig_raw.isin(false_tokens), 0)
    unknown = ig_raw[mapped.isna()].dropna().unique().tolist()
    if len(unknown) == 2:
        # deterministic: map lexicographically smaller to 0, larger to 1
        a, b = sorted(unknown)
        auto_map = {a: 0, b: 1}
        print("ℹ️ intervention_group auto-map used:", auto_map)
        mapped = mapped.fillna(ig_raw.map(auto_map))
    df_smokers["intervention_group"] = mapped

# Drop rows where intervention_group is NA after mapping
before = len(df_smokers)
df_smokers = df_smokers.dropna(subset=["intervention_group"]).copy()
after = len(df_smokers)
if after == 0:
    raise ValueError("All rows have NA intervention_group after mapping; inspect raw values.")
if after < before:
    print(f"ℹ️ Dropped {before - after} rows with NA intervention_group.")
df_smokers["intervention_group"] = df_smokers["intervention_group"].astype(int)

# ----------------------------
# Simulate quit outcome (replace with true outcome if available)
# ----------------------------
np.random.seed(42)
df_smokers["quit"] = np.where(
    df_smokers["intervention_group"] == 1,
    np.random.binomial(1, 0.45, size=len(df_smokers)),
    np.random.binomial(1, 0.25, size=len(df_smokers))
).astype(int)
print("🎯 Quit counts:\n", df_smokers["quit"].value_counts())

# ----------------------------
# Age groups & race ethnicity categories
# ----------------------------
age_bins = [17, 29, 39, 49, 59, 69, 79, 120]
age_labels = ["18–29", "30–39", "40–49", "50–59", "60–69", "70–79", "80+"]
df_smokers["age_group"] = pd.cut(
    df_smokers["age"],
    bins=age_bins,
    labels=age_labels,
    include_lowest=True,
    right=True
)

df_smokers["race_ethnicity"] = df_smokers["race_ethnicity"].astype("category")

# ----------------------------
# Build model dataset & drop NA in model vars
# ----------------------------
model_vars = ["quit", "intervention_group", "age_group", "race_ethnicity"]
model_df = df_smokers[model_vars].dropna().copy()

if model_df.empty:
    print("🔎 Diagnostics:")
    for c in model_vars:
        print(f" - {c} nulls:", df_smokers[c].isna().sum())
    raise ValueError("After dropping missing values in model variables, no rows remain.")

# Ensure both outcome classes present
if model_df["quit"].nunique() < 2:
    raise ValueError("Outcome 'quit' has a single class after filtering; cannot fit logistic regression.")

# ----------------------------
# Reference groups
# ----------------------------
# Age reference: prefer 18–29 if present
if isinstance(model_df["age_group"].dtype, CategoricalDtype):
    # remove unused, then set order to the intersection with desired order
    model_df["age_group"] = model_df["age_group"].cat.remove_unused_categories()
    present = [lvl for lvl in age_labels if lvl in model_df["age_group"].cat.categories]
    model_df["age_group"] = model_df["age_group"].cat.set_categories(present, ordered=True)
    ref_age = "18–29" if "18–29" in present else present[0]
else:
    # non-categorical fallback
    present = sorted(model_df["age_group"].astype(str).unique().tolist())
    ref_age = "18–29" if "18–29" in present else present[0]

# Race reference = most frequent
ref_race = model_df["race_ethnicity"].mode().iat[0]

# ----------------------------
# Logistic regression
# ----------------------------
formula = (
    f"quit ~ intervention_group"
    f" + C(age_group, Treatment(reference='{ref_age}'))"
    f" + C(race_ethnicity, Treatment(reference='{ref_race}'))"
)
model = smf.logit(formula, data=model_df).fit(disp=False)

# ----------------------------
# Build OR table
# ----------------------------
sum_table = model.summary2().tables[1].copy()
sum_table = sum_table.reset_index().rename(columns={"index": "Term"})
sum_table = sum_table[sum_table["Term"] != "Intercept"].copy()

sum_table["OR"] = np.exp(sum_table["Coef."])
sum_table["CI Lower"] = np.exp(sum_table["Coef."] - 1.96 * sum_table["Std.Err."])
sum_table["CI Upper"] = np.exp(sum_table["Coef."] + 1.96 * sum_table["Std.Err."])

def pretty_label(term: str) -> str:
    if term == "intervention_group":
        return "Cessation Program (vs Usual Care)"
    m_age = re.match(r"C\(age_group, Treatment\(reference=.*\)\)\[T\.(.+)\]", term)
    if m_age:
        return f"Age: {m_age.group(1)} (vs {ref_age})"
    m_race = re.match(r"C\(race_ethnicity, Treatment\(reference=.*\)\)\[T\.(.+)\]", term)
    if m_race:
        return f"Race/Ethnicity: {m_race.group(1)} (vs {ref_race})"
    return term

sum_table["Variable"] = sum_table["Term"].apply(pretty_label)
sum_table = sum_table.sort_values("OR", ascending=True).reset_index(drop=True)

# ----------------------------
# Forest plot (matplotlib)
# ----------------------------
plt.figure(figsize=(9, max(4, 0.48 * len(sum_table))))
y = np.arange(len(sum_table))
x = sum_table["OR"].values
xerr = np.vstack([x - sum_table["CI Lower"].values, sum_table["CI Upper"].values - x])

plt.errorbar(
    x=x,
    y=y,
    xerr=xerr,
    fmt='o',
    ecolor='black',
    capsize=3,
    elinewidth=1,
    markersize=5,
    color='black'
)
plt.yticks(y, sum_table["Variable"])
plt.axvline(1.0, linestyle="--", linewidth=1, color="gray")
plt.xscale("log")
plt.xlabel("Odds Ratio (log scale)")
plt.title("Adjusted Odds Ratios for Quit Success\nIntervention, Age Groups, Race/Ethnicity")
plt.tight_layout()

out_path = FIGURE_DIR / "forest_or_age_race.png"
plt.savefig(out_path, dpi=300, bbox_inches="tight")
plt.close()

print(f"✅ Forest plot saved to: {out_path.resolve()}")
print(f"Ref groups → age_group: {ref_age} | race_ethnicity: {ref_race}")


✅ Loaded: C:\Users\hayde\Desktop\simulated-smoking-cessation-cohort\data\processed_simulated_smoking_cessation_cohort.csv
📋 Columns: ['id', 'education', 'income', 'disability', 'employment', 'metro', 'age', 'sex', 'race_ethnicity', 'sex_female', 'education_code', 'income_code', 'race_Black', 'race_Hispanic', 'race_Other', 'race_White', 'baseline_smoker', 'intervention_group']
ℹ️ intervention_group mapping (forced): {'cessation program': 1, 'usual care': 0}
🎯 Quit counts:
 quit
0    1311
1     675
Name: count, dtype: int64
✅ Forest plot saved to: C:\Users\hayde\Desktop\simulated-smoking-cessation-cohort\figures\forest_or_age_race.png
Ref groups → age_group: 18–29 | race_ethnicity: White
